In [1]:
import pandas as pd
import numpy as np
from prophet import Prophet
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpInteger

# ====================
# 1. Demand Forecasting
# ====================

# Read demand.csv with headers: Year, Size, Distance, Demand (km)
demand_df = pd.read_csv("data/demand.csv")

# Forecast demand per (Size, Distance) combination using Prophet.
forecast_horizon = list(range(2023, 2039))
forecast_demand = {}  # dict with key: (Size, Distance) -> forecast DataFrame

for (size, distance), group in demand_df.groupby(["Size", "Distance"]):
    # Use 'Year' as date and 'Demand (km)' as the demand value
    df_prophet = group[['Year', 'Demand (km)']].rename(columns={'Year': 'ds', 'Demand (km)': 'y'})
    # Convert year to datetime (using Jan 1 as date)
    df_prophet['ds'] = pd.to_datetime(df_prophet['ds'].astype(str) + '-01-01')
    
    model = Prophet(yearly_seasonality=False, daily_seasonality=False)
    model.fit(df_prophet)
    
    # Create dataframe for future predictions from 2023 to 2038
    future = pd.DataFrame({'ds': pd.to_datetime([str(year) + '-01-01' for year in forecast_horizon])})
    forecast = model.predict(future)
    forecast_df = forecast[['ds', 'yhat']].rename(columns={'ds': 'Year'})
    forecast_df['Year'] = forecast_df['Year'].dt.year
    forecast_demand[(size, distance)] = forecast_df

# ====================
# 2. Fleet Optimization Model Setup
# ====================

# Load other datasets
vehicles_df = pd.read_csv("data/vehicles.csv")  # columns: ID, Vehicle, Size, Year, Cost, Yearly_range, Distance
fuels_df = pd.read_csv("data/fuels.csv")        # columns: Fuel, Year, Emissions, Cost, Cost_Uncertainty
carbon_limits_df = pd.read_csv("data/carbon_emissions.csv")  # columns: Year, Carbon emission CO2/kg

# Assume vehicle fuel consumption, insurance, maintenance, etc. are pre-processed as needed.
# Define indices and sets
years = list(range(2023, 2039))
vehicles = vehicles_df['ID'].unique()
vehicle_props = vehicles_df.set_index("ID").to_dict(orient="index")

# Decision variables:
# buy[year, vehicle] = number of vehicles bought in that year for that model (only if model year equals that year)
buy = {(yr, vid): LpVariable(f"buy_{yr}_{vid}", lowBound=0, cat=LpInteger)
       for yr in years for vid in vehicles if vehicle_props[vid]['Year'] == yr}
# use[year, vehicle] = number of vehicles used in that year (should not exceed fleet available)
use = {(yr, vid): LpVariable(f"use_{yr}_{vid}", lowBound=0, cat=LpInteger)
       for yr in years for vid in vehicles}
# sell[year, vehicle] = vehicles sold at end of that year
sell = {(yr, vid): LpVariable(f"sell_{yr}_{vid}", lowBound=0, cat=LpInteger)
        for yr in years for vid in vehicles}

# Allocation: assign vehicles to meet demand for each (Size, Distance) bucket
alloc = {(yr, s, d, vid): LpVariable(f"alloc_{yr}_{s}_{d}_{vid}", lowBound=0, cat=LpInteger)
         for yr in years
         for (s, d) in forecast_demand.keys()
         for vid in vehicles if vehicle_props[vid]['Size'] == s}

# Create the LP problem:
prob = LpProblem("Fleet_Optimization", LpMinimize)

# ===============
# 3. Objective Function
# ===============
purchase_cost = lpSum(buy[(yr, vid)] * vehicle_props[vid]['Cost'] for (yr, vid) in buy)
resale_value = lpSum(sell[(yr, vid)] * 0.3 * vehicle_props[vid]['Cost']
                     for (yr, vid) in sell)
fuel_cost = lpSum(alloc[(yr, s, d, vid)] * 0.5  # placeholder cost per km
                  for (yr, s, d, vid) in alloc)

prob += (purchase_cost + fuel_cost - resale_value), "Total_Fleet_Cost"

# ===============
# 4. Constraints
# ===============

# Constraint 1: Demand satisfaction for each (Size, Distance) bucket and year
for (s, d), df_forecast in forecast_demand.items():
    for yr in years:
        demand_val = df_forecast.loc[df_forecast['Year'] == yr, 'yhat']
        if not demand_val.empty:
            prob += (lpSum(alloc[(yr, s, d, vid)] * vehicle_props[vid]['Yearly_range']
                           for vid in vehicles if vehicle_props[vid]['Size'] == s)
                     >= float(demand_val), f"Demand_{yr}_{s}_{d}")

# Constraint 2: Distance bucket matching
for yr in years:
    for (s, d), _ in forecast_demand.items():
        for vid in vehicles:
            if vehicle_props[vid]['Size'] == s:
                vehicle_bucket = vehicle_props[vid]['Distance']
                bucket_order = {'D1': 1, 'D2': 2, 'D3': 3, 'D4': 4}
                if bucket_order[vehicle_bucket] < bucket_order[d]:
                    prob += (alloc[(yr, s, d, vid)] == 0, f"Distance_{yr}_{s}_{d}_{vid}")

# Constraint 3: Fleet availability and lifetime (10-year life)
# fleet = {}
# for vid in vehicles:
#     for yr in years:
#         active_fleet = lpSum(buy[(y, vid)] for y in years if y <= yr and yr - y < 10) - \
#                        lpSum(sell[(y, vid)] for y in years if y < yr)
#         fleet[(yr, vid)] = active_fleet
#         prob += (use[(yr, vid)] <= active_fleet, f"Fleet_use_{yr}_{vid}")


# Constraint 3: Fleet availability and lifetime (10-year life)
fleet = {}
for vid in vehicles:
    for yr in years:
        active_fleet = lpSum(buy[(y, vid)] for y in years if y <= yr and (y, vid) in buy and yr - y < 10) - \
                       lpSum(sell[(y, vid)] for y in years if y < yr)
        fleet[(yr, vid)] = active_fleet
        prob += (use[(yr, vid)] <= active_fleet, f"Fleet_use_{yr}_{vid}")


# Constraint 4: Selling limits – at most 20% of the fleet can be sold in any year.
for yr in years:
    total_fleet = lpSum(fleet[(yr, vid)] for vid in vehicles)
    total_sell = lpSum(sell[(yr, vid)] for vid in vehicles)
    prob += (total_sell <= 0.20 * total_fleet, f"Sell_limit_{yr}")

# Constraint 5: Vehicle model purchase year constraint
for (yr, vid) in buy:
    if yr != vehicle_props[vid]['Year']:
        prob += (buy[(yr, vid)] == 0, f"Purchase_year_{yr}_{vid}")

# Constraint 6: Linking allocation with usage
for yr in years:
    for vid in vehicles:
        relevant_alloc = lpSum(alloc[(yr, s, d, vid)] for (s, d) in forecast_demand.keys() if vehicle_props[vid]['Size'] == s)
        prob += (relevant_alloc <= use[(yr, vid)], f"Allocation_use_{yr}_{vid}")

# Constraint 7: Carbon emissions constraint (placeholder)
for yr in years:
    total_emission = lpSum(alloc[(yr, s, d, vid)] * 0.2  # placeholder emission per km
                           for (s, d) in forecast_demand.keys() 
                           for vid in vehicles if vehicle_props[vid]['Size'] == s)
    carbon_limit = float(carbon_limits_df.loc[carbon_limits_df['Year'] == yr, 'Carbon emission CO2/kg'])
    prob += (total_emission <= carbon_limit, f"Carbon_limit_{yr}")

# [Additional constraints would go here.]

# ====================
# 5. Solve the Model
# ====================

print("Starting optimization...")
prob.solve()
print("Optimization complete. Status:", prob.status)

# ====================
# 6. Extract and Save Results
# ====================

results = []
for yr in years:
    for vid in vehicles:
        b = buy.get((yr, vid))
        u = use.get((yr, vid))
        s = sell.get((yr, vid))
        if b is not None and b.varValue and b.varValue > 0:
            results.append({'Year': yr, 'ID': vid, 'Num_Vehicles': int(b.varValue), 'Type': 'Buy', 'Fuel': None,
                            'Distance': vehicle_props[vid]['Distance'],
                            'Distance_per_vehicle(km)': vehicle_props[vid]['Yearly_range']})
        if u is not None and u.varValue and u.varValue > 0:
            results.append({'Year': yr, 'ID': vid, 'Num_Vehicles': int(u.varValue), 'Type': 'Use', 'Fuel': None,
                            'Distance': vehicle_props[vid]['Distance'],
                            'Distance_per_vehicle(km)': vehicle_props[vid]['Yearly_range']})
        if s is not None and s.varValue and s.varValue > 0:
            results.append({'Year': yr, 'ID': vid, 'Num_Vehicles': int(s.varValue), 'Type': 'Sell', 'Fuel': None,
                            'Distance': vehicle_props[vid]['Distance'],
                            'Distance_per_vehicle(km)': vehicle_props[vid]['Yearly_range']})

alloc_results = []
for (yr, s, d, vid), var in alloc.items():
    if var.varValue and var.varValue > 0:
        alloc_results.append({'Year': yr, 'ID': vid, 'Num_Vehicles': int(var.varValue),
                              'Type': 'Use', 'Fuel': None, 'Distance': d,
                              'Distance_per_vehicle(km)': vehicle_props[vid]['Yearly_range']})

results_df = pd.DataFrame(results)
alloc_df = pd.DataFrame(alloc_results)

results_df.to_csv("fleet_operations_new.csv", index=False)
alloc_df.to_csv("demand_allocation_new.csv", index=False)

print("Results saved.")


11:33:00 - cmdstanpy - INFO - Chain [1] start processing
11:33:00 - cmdstanpy - INFO - Chain [1] done processing
11:33:00 - cmdstanpy - INFO - Chain [1] start processing
11:33:00 - cmdstanpy - INFO - Chain [1] done processing
11:33:00 - cmdstanpy - INFO - Chain [1] start processing
11:33:00 - cmdstanpy - INFO - Chain [1] done processing
11:33:01 - cmdstanpy - INFO - Chain [1] start processing
11:33:01 - cmdstanpy - INFO - Chain [1] done processing
11:33:01 - cmdstanpy - INFO - Chain [1] start processing
11:33:01 - cmdstanpy - INFO - Chain [1] done processing
11:33:01 - cmdstanpy - INFO - Chain [1] start processing
11:33:01 - cmdstanpy - INFO - Chain [1] done processing
11:33:02 - cmdstanpy - INFO - Chain [1] start processing
11:33:02 - cmdstanpy - INFO - Chain [1] done processing
11:33:02 - cmdstanpy - INFO - Chain [1] start processing
11:33:02 - cmdstanpy - INFO - Chain [1] done processing
11:33:02 - cmdstanpy - INFO - Chain [1] start processing
11:33:02 - cmdstanpy - INFO - Chain [1]

Starting optimization...
Optimization complete. Status: 1
Results saved.


In [3]:
import random
import numpy as np
import pandas as pd
from deap import base, creator, tools, algorithms

# =============================================================================
# Data Loading and Pre-Processing
# =============================================================================
# For demonstration purposes, we assume that you have pre-loaded data frames:
# - demand_df (with columns: Year, Size, Distance, Demand (km))
# - vehicles_df (with columns: ID, Size, Year, Cost, Yearly_range, Distance, FuelType, Emission_rate)
# - carbon_limits_df (with columns: Year, Carbon emission CO2/kg)
#
# You will need to create appropriate dictionaries or data structures from these.
#
# For example, we build a dictionary for vehicle properties:
vehicles_df = pd.read_csv("data/vehicles.csv")
vehicle_props = vehicles_df.set_index("ID").to_dict(orient="index")

# Similarly, load carbon limits (indexed by Year) and demand forecasts.
carbon_limits_df = pd.read_csv("data/carbon_emissions.csv")
carbon_limits = carbon_limits_df.set_index("Year")["Carbon emission CO2/kg"].to_dict()

# For demand, suppose we have a nested dict: demand[(year, size, distance)] = km
demand_df = pd.read_csv("data/demand.csv")
demand = {}
for idx, row in demand_df.iterrows():
    key = (int(row["Year"]), row["Size"], row["Distance"])
    demand[key] = float(row["Demand (km)"])

# List of years in planning horizon
years = list(range(2023, 2039))
# List of vehicles available by model year (only available in their designated purchase year)
available_vehicle_ids = list(vehicle_props.keys())

# =============================================================================
# Evolutionary Algorithm Setup
# =============================================================================
# For multi-objective optimization, we want to minimize:
#  - Overall fleet cost (purchase, fuel, maintenance, minus resale)
#  - Total emissions penalty (if emissions exceed limits, we penalize)
# We assume other operational constraints (like demand satisfaction, fleet lifetime, sell limits)
# are incorporated via penalty functions.
#
# We design a solution representation as follows:
#  - An individual is a dictionary mapping (year, vehicle ID) to the number purchased.
#  - From purchases, we will simulate fleet evolution (usage and sales) over years.
#
# In this example, we represent an individual as a list of integers, where each gene corresponds
# to the number of vehicles purchased for a particular (year, vehicle ID) pair.
#
# We first create an ordering list for these pairs.
solution_keys = []
for yr in years:
    for vid in available_vehicle_ids:
        # Only allow purchase if the vehicle's designated model year is equal to the current year.
        if vehicle_props[vid]['Year'] == yr:
            solution_keys.append((yr, vid))

# Number of decision variables:
n_vars = len(solution_keys)

# Define our multi-objective fitness: minimize cost and emissions penalty.
creator.create("FitnessMulti", base.Fitness, weights=(-1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMulti)

# =============================================================================
# Evaluation Function with Constraints as Penalties
# =============================================================================
def evaluate(individual):
    # Build the purchase dictionary from the individual
    # individual is a list of non-negative integers corresponding to solution_keys
    purchase_plan = { key: individual[i] for i, key in enumerate(solution_keys) }
    
    # Simulate fleet evolution:
    # For each year, compute the available fleet from purchases (taking into account lifetime of 10 years)
    fleet = {yr: {} for yr in years}
    for yr in years:
        for vid in available_vehicle_ids:
            # Sum purchases in eligible years
            total = 0
            for y in years:
                if y <= yr and yr - y < 10 and (y, vid) in purchase_plan:
                    total += purchase_plan[(y, vid)]
            fleet[yr][vid] = total

    # Compute total purchase cost
    purchase_cost = 0.0
    for (yr, vid), num in purchase_plan.items():
        purchase_cost += num * vehicle_props[vid]['Cost']
    
    # Compute a simplified fuel cost:
    # Assume cost per km and assign the vehicle's full yearly range usage.
    fuel_cost = 0.0
    for yr in years:
        for vid, count in fleet[yr].items():
            # For simplicity, assume each vehicle is used to cover its yearly range.
            fuel_cost += count * vehicle_props[vid]['Yearly_range'] * 0.5  # 0.5 is a placeholder
    
    # Compute a simplified resale value (assuming a 30% resale value at sale)
    resale_value = 0.0
    # (For demonstration, we do not simulate sales in detail.)
    
    total_cost = purchase_cost + fuel_cost - resale_value

    # Compute total emissions over the planning horizon
    total_emissions = 0.0
    for yr in years:
        yearly_emission = 0.0
        for vid, count in fleet[yr].items():
            # Assume each vehicle emits (Yearly_range * Emission_rate) per year.
            yearly_emission += count * vehicle_props[vid]['Yearly_range'] * vehicle_props[vid]['Emission_rate']
        total_emissions += yearly_emission

    # Constraint: Demand satisfaction
    # For each (year, size, distance), ensure that total km provided by vehicles of that size is at least demand.
    demand_penalty = 0.0
    for key, req in demand.items():
        yr, size, distance = key
        provided = 0.0
        # Sum the capacity of vehicles of the given size that can cover the distance bucket.
        for vid, props in vehicle_props.items():
            if props['Size'] == size:
                # Check if the vehicle's distance rating meets the bucket requirement.
                bucket_order = {'D1': 1, 'D2': 2, 'D3': 3, 'D4': 4}
                if bucket_order[props['Distance']] >= bucket_order[distance]:
                    provided += fleet.get(yr, {}).get(vid, 0) * props['Yearly_range']
        # If provided is less than demand, add a penalty proportional to the shortfall.
        if provided < req:
            demand_penalty += (req - provided) * 1000  # Penalty multiplier

    # Constraint: Carbon emissions limit
    emission_penalty = 0.0
    for yr in years:
        yearly_emission = 0.0
        for vid, count in fleet[yr].items():
            yearly_emission += count * vehicle_props[vid]['Yearly_range'] * vehicle_props[vid]['Emission_rate']
        limit = carbon_limits.get(yr, np.inf)
        if yearly_emission > limit:
            emission_penalty += (yearly_emission - limit) * 1000  # Penalty multiplier

    # Additional operational constraints (fleet lifetime, sell limits, etc.) can be added as extra penalties.
    # For now, we add a total penalty:
    total_penalty = demand_penalty + emission_penalty

    # Multi-objective: minimize total cost and total penalty (emissions/demand shortfall)
    return total_cost, total_penalty

# =============================================================================
# Toolbox Setup for Evolutionary Algorithm
# =============================================================================
toolbox = base.Toolbox()
# Attribute: each gene is an integer, say between 0 and a maximum purchase count (e.g., 100)
toolbox.register("attr_int", random.randint, 0, 100)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, n=n_vars)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=100, indpb=0.1)
toolbox.register("select", tools.selNSGA2)

# =============================================================================
# Run the Evolutionary Algorithm
# =============================================================================
def main():
    random.seed(42)
    pop = toolbox.population(n=200)
    NGEN = 50
    MU = 200
    LAMBDA = 100
    CXPB = 0.7
    MUTPB = 0.2

    # Use the NSGA-II algorithm for multi-objective optimization
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean, axis=0)
    stats.register("min", np.min, axis=0)
    
    hof = tools.ParetoFront()
    
    algorithms.eaMuPlusLambda(pop, toolbox, mu=MU, lambda_=LAMBDA, cxpb=CXPB, mutpb=MUTPB,
                                ngen=NGEN, stats=stats, halloffame=hof, verbose=True)
    
    return pop, hof

if __name__ == "__main__":
    population, pareto_front = main()
    
    # Save Pareto front results
    results = []
    for ind in pareto_front:
        sol = { key: ind[i] for i, key in enumerate(solution_keys) }
        fitness = ind.fitness.values
        results.append({"Solution": sol, "Total_Cost": fitness[0], "Penalty": fitness[1]})
    
    results_df = pd.DataFrame(results)
    results_df.to_csv("pareto_front_results.csv", index=False)
    print("Pareto front saved.")


KeyError: 'Emission_rate'

In [11]:
import random
import numpy as np
import pandas as pd
from deap import base, creator, tools, algorithms

# =============================================================================
# Data Loading and Pre-Processing
# =============================================================================

# Load vehicles data
vehicles_df = pd.read_csv("data/vehicles.csv")
# Expected columns in vehicles.csv: ID, Vehicle, Size, Year, Cost, Yearly_range, Distance, [other columns...]

# Load vehicles_fuel data
vehicles_fuels_df = pd.read_csv("data/vehicles_fuels.csv")
# Expected columns: ID, Fuel, Consumption (unit_fuel/km)

# Load fuel emissions data
fuels_df = pd.read_csv("data/fuels.csv")
# Expected columns: Fuel,Year,emissions (CO2/unit_fuel),Cost ($/unit_fuel),cost Uncertainty (±%)

# For computing emission rates, we need to decide on which fuel year to use.
# Here, we take the first available value per fuel type.
fuels_selected = fuels_df.sort_values("Year").groupby("Fuel").first().reset_index()

# Merge vehicles_fuels_df with fuels_selected on Fuel.
vehicles_fuels_merged = vehicles_fuels_df.merge(
    fuels_selected[['Fuel', 'emissions (CO2/unit_fuel)']],
    on="Fuel", how="left"
)

# Calculate Emission_rate for each vehicle (CO2/km)
vehicles_fuels_merged["Emission_rate"] = (
    vehicles_fuels_merged["Consumption (unit_fuel/km)"] *
    vehicles_fuels_merged["emissions (CO2/unit_fuel)"]
)

# Merge the computed emission rate into vehicles_df based on ID.
vehicles_df = vehicles_df.merge(
    vehicles_fuels_merged[["ID", "Emission_rate"]],
    on="ID", how="left"
)

# Resolve duplicate IDs: Option A (remove duplicates) – keep first occurrence.
vehicles_df = vehicles_df.drop_duplicates(subset="ID")

# Build a dictionary of vehicle properties.
vehicle_props = vehicles_df.set_index("ID").to_dict(orient="index")

# Load carbon emissions limits
carbon_limits_df = pd.read_csv("data/carbon_emissions.csv")
# Expected columns: Year, Carbon emission CO2/kg
carbon_limits = carbon_limits_df.set_index("Year")["Carbon emission CO2/kg"].to_dict()

# Load demand data
demand_df = pd.read_csv("data/demand.csv")
# Expected columns: Year, Size, Distance, Demand (km)
demand = {}
for idx, row in demand_df.iterrows():
    key = (int(row["Year"]), row["Size"], row["Distance"])
    demand[key] = float(row["Demand (km)"])

# Define planning horizon and available vehicles.
years = list(range(2023, 2039))
available_vehicle_ids = list(vehicle_props.keys())

# =============================================================================
# Evolutionary Algorithm Setup
# =============================================================================

# An individual represents a purchase plan.
# We allow purchases only in the designated model year.
solution_keys = []
for yr in years:
    for vid in available_vehicle_ids:
        if vehicle_props[vid]['Year'] == yr:
            solution_keys.append((yr, vid))
n_vars = len(solution_keys)

# Define multi-objective fitness: minimize total cost and penalty.
creator.create("FitnessMulti", base.Fitness, weights=(-1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMulti)

# =============================================================================
# Evaluation Function with Penalties for Constraints
# =============================================================================
def evaluate(individual):
    # Build purchase plan dictionary from individual.
    purchase_plan = { key: individual[i] for i, key in enumerate(solution_keys) }
    
    # Simulate fleet evolution with a 10-year lifetime.
    fleet = {yr: {} for yr in years}
    for yr in years:
        for vid in available_vehicle_ids:
            total = 0
            for y in years:
                if y <= yr and (y, vid) in purchase_plan and (yr - y) < 10:
                    total += purchase_plan[(y, vid)]
            fleet[yr][vid] = total

    # Compute total purchase cost.
    purchase_cost = sum(
        purchase_plan[(yr, vid)] * vehicle_props[vid]['Cost']
        for (yr, vid) in purchase_plan
    )
    
    # Compute simplified fuel cost (assume full yearly range usage at a placeholder cost per km).
    fuel_cost = 0.0
    for yr in years:
        for vid, count in fleet[yr].items():
            fuel_cost += count * vehicle_props[vid]['Yearly_range'] * 0.5  # placeholder cost/km

    # Simplified resale value (not simulated in detail).
    resale_value = 0.0
    total_cost = purchase_cost + fuel_cost - resale_value

    # Compute total emissions.
    total_emissions = 0.0
    for yr in years:
        yearly_emission = 0.0
        for vid, count in fleet[yr].items():
            # Each vehicle emits: count * Yearly_range * Emission_rate (CO2/km)
            yearly_emission += count * vehicle_props[vid]['Yearly_range'] * vehicle_props[vid]['Emission_rate']
        total_emissions += yearly_emission

    # Constraint: Demand satisfaction.
    demand_penalty = 0.0
    bucket_order = {'D1': 1, 'D2': 2, 'D3': 3, 'D4': 4}
    for key, req in demand.items():
        yr, size, distance = key
        provided = 0.0
        for vid, props in vehicle_props.items():
            if props['Size'] == size:
                if bucket_order[props['Distance']] >= bucket_order[distance]:
                    provided += fleet.get(yr, {}).get(vid, 0) * props['Yearly_range']
        if provided < req:
            demand_penalty += (req - provided) * 1000  # penalty multiplier

    # Constraint: Carbon emissions limit.
    emission_penalty = 0.0
    for yr in years:
        yearly_emission = 0.0
        for vid, count in fleet[yr].items():
            yearly_emission += count * vehicle_props[vid]['Yearly_range'] * vehicle_props[vid]['Emission_rate']
        limit = carbon_limits.get(yr, np.inf)
        if yearly_emission > limit:
            emission_penalty += (yearly_emission - limit) * 1000  # penalty multiplier

    total_penalty = demand_penalty + emission_penalty

    return total_cost, total_penalty

# =============================================================================
# Toolbox Setup for Evolutionary Algorithm
# =============================================================================
toolbox = base.Toolbox()
toolbox.register("attr_int", random.randint, 0, 100)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, n=n_vars)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=100, indpb=0.1)
toolbox.register("select", tools.selNSGA2)

# =============================================================================
# Run the Evolutionary Algorithm (NSGA-II)
# =============================================================================
def main():
    random.seed(42)
    pop = toolbox.population(n=200)
    NGEN = 50
    MU = 200
    LAMBDA = 100
    CXPB = 0.7
    MUTPB = 0.2

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean, axis=0)
    stats.register("min", np.min, axis=0)
    
    hof = tools.ParetoFront()
    
    algorithms.eaMuPlusLambda(pop, toolbox, mu=MU, lambda_=LAMBDA, cxpb=CXPB, mutpb=MUTPB,
                               ngen=NGEN, stats=stats, halloffame=hof, verbose=True)
    
    return pop, hof

if __name__ == "__main__":
    population, pareto_front = main()
    
    # Save Pareto front results.
    results = []
    for ind in pareto_front:
        sol = { key: ind[i] for i, key in enumerate(solution_keys) }
        fitness = ind.fitness.values
        results.append({"Solution": sol, "Total_Cost": fitness[0], "Penalty": fitness[1]})
    
    results_df = pd.DataFrame(results)
    results_df.to_csv("pareto_front_results.csv", index=False)
    print("Pareto front saved.")


c:\Users\hjain\anaconda3\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMulti' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\hjain\anaconda3\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals	avg                            	min                            
0  	200   	[5.19549650e+09 2.36772125e+12]	[4.49131771e+09 1.87174123e+12]
1  	92    	[5.06064714e+09 2.28928017e+12]	[4.38150721e+09 1.87174123e+12]
2  	91    	[4.99271007e+09 2.23860186e+12]	[4.38150721e+09 1.87174123e+12]
3  	92    	[4.93259707e+09 2.19946433e+12]	[4.38150721e+09 1.87174123e+12]
4  	87    	[4.87377792e+09 2.16286620e+12]	[4.29110492e+09 1.87174123e+12]
5  	89    	[4.81665485e+09 2.12131623e+12]	[4.29110492e+09 1.87174123e+12]
6  	91    	[4.77342226e+09 2.08522781e+12]	[4.29110492e+09 1.87174123e+12]
7  	92    	[4.71013536e+09 2.05052578e+12]	[4.24862213e+09 1.87174123e+12]
8  	92    	[4.63110835e+09 2.01726313e+12]	[4.11838694e+09 1.85819344e+12]
9  	90    	[4.57292308e+09 1.98763628e+12]	[4.11838694e+09 1.82765122e+12]
10 	88    	[4.5314987e+09 1.9641018e+12]  	[4.11838694e+09 1.79220771e+12]
11 	89    	[4.49098716e+09 1.93916912e+12]	[4.11838694e+09 1.79220771e+12]
12 	87    	[4.44509310e+0

In [13]:
import random
import numpy as np
import pandas as pd
from deap import base, creator, tools, algorithms

# =============================================================================
# Data Loading and Pre-Processing
# =============================================================================

# Load vehicles data
vehicles_df = pd.read_csv("data/vehicles.csv")
# Expected columns: ID, Vehicle, Size, Year, Cost, Yearly_range, Distance, FuelType (if available)

# Load vehicles_fuels data
vehicles_fuels_df = pd.read_csv("data/vehicles_fuels.csv")
# Expected columns: ID, Fuel, Consumption (unit_fuel/km)

# Load fuel emissions data
fuels_df = pd.read_csv("data/fuels.csv")
# Expected columns: Fuel, Year, emissions (CO2/unit_fuel), Cost ($/unit_fuel), cost Uncertainty (±%)

# For computing emission rates, we select the first available row per Fuel type.
fuels_selected = fuels_df.sort_values("Year").groupby("Fuel").first().reset_index()

# Merge vehicles_fuels_df with fuels_selected on Fuel.
vehicles_fuels_merged = vehicles_fuels_df.merge(
    fuels_selected[['Fuel', 'emissions (CO2/unit_fuel)']],
    on="Fuel", how="left"
)

# Calculate Emission_rate (CO2 per km)
vehicles_fuels_merged["Emission_rate"] = (
    vehicles_fuels_merged["Consumption (unit_fuel/km)"] *
    vehicles_fuels_merged["emissions (CO2/unit_fuel)"]
)

# Merge computed emission rate into vehicles_df based on ID.
vehicles_df = vehicles_df.merge(
    vehicles_fuels_merged[["ID", "Emission_rate"]],
    on="ID", how="left"
)

# Resolve duplicate IDs by dropping duplicates (keep first occurrence)
vehicles_df = vehicles_df.drop_duplicates(subset="ID")

# Build dictionary of vehicle properties.
vehicle_props = vehicles_df.set_index("ID").to_dict(orient="index")

# Load carbon emissions limits.
carbon_limits_df = pd.read_csv("data/carbon_emissions.csv")
# Expected columns: Year, Carbon emission CO2/kg
carbon_limits = carbon_limits_df.set_index("Year")["Carbon emission CO2/kg"].to_dict()

# Load demand data.
demand_df = pd.read_csv("data/demand.csv")
# Expected columns: Year, Size, Distance, Demand (km)
demand = {}
for idx, row in demand_df.iterrows():
    key = (int(row["Year"]), row["Size"], row["Distance"])
    demand[key] = float(row["Demand (km)"])

# Define planning horizon and available vehicles.
years = list(range(2023, 2039))
available_vehicle_ids = list(vehicle_props.keys())

# =============================================================================
# Evolutionary Algorithm Setup
# =============================================================================

# Each individual represents a purchase plan.
# We only allow purchases in the designated model year.
solution_keys = []
for yr in years:
    for vid in available_vehicle_ids:
        if vehicle_props[vid]['Year'] == yr:
            solution_keys.append((yr, vid))
n_vars = len(solution_keys)

# Create multi-objective fitness: minimize (total_cost, penalty)
creator.create("FitnessMulti", base.Fitness, weights=(-1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMulti)

def evaluate(individual):
    # Build purchase plan from individual (list of integers)
    purchase_plan = { key: individual[i] for i, key in enumerate(solution_keys) }
    
    # Simulate fleet evolution (10-year lifetime)
    fleet = {yr: {} for yr in years}
    for yr in years:
        for vid in available_vehicle_ids:
            total = 0
            for y in years:
                if y <= yr and (y, vid) in purchase_plan and (yr - y) < 10:
                    total += purchase_plan[(y, vid)]
            fleet[yr][vid] = total

    # Compute total purchase cost.
    purchase_cost = sum(
        purchase_plan[(yr, vid)] * vehicle_props[vid]['Cost']
        for (yr, vid) in purchase_plan
    )
    
    # Compute simplified fuel cost (assume full yearly range usage at placeholder cost 0.5 per km)
    fuel_cost = 0.0
    for yr in years:
        for vid, count in fleet[yr].items():
            fuel_cost += count * vehicle_props[vid]['Yearly_range'] * 0.5

    # (Simplified resale value is assumed zero.)
    total_cost = purchase_cost + fuel_cost

    # Compute total emissions (each vehicle: Yearly_range * Emission_rate)
    total_emissions = 0.0
    for yr in years:
        yearly_emission = 0.0
        for vid, count in fleet[yr].items():
            yearly_emission += count * vehicle_props[vid]['Yearly_range'] * vehicle_props[vid]['Emission_rate']
        total_emissions += yearly_emission

    # Demand satisfaction penalty:
    demand_penalty = 0.0
    bucket_order = {'D1': 1, 'D2': 2, 'D3': 3, 'D4': 4}
    for key, req in demand.items():
        d_year, d_size, d_distance = key
        provided = 0.0
        for vid, props in vehicle_props.items():
            if props['Size'] == d_size:
                if bucket_order[props['Distance']] >= bucket_order[d_distance]:
                    provided += fleet.get(d_year, {}).get(vid, 0) * props['Yearly_range']
        if provided < req:
            demand_penalty += (req - provided) * 1000  # Penalty multiplier

    # Carbon emissions penalty:
    emission_penalty = 0.0
    for yr in years:
        yearly_emission = 0.0
        for vid, count in fleet[yr].items():
            yearly_emission += count * vehicle_props[vid]['Yearly_range'] * vehicle_props[vid]['Emission_rate']
        limit = carbon_limits.get(yr, np.inf)
        if yearly_emission > limit:
            emission_penalty += (yearly_emission - limit) * 1000  # Penalty multiplier

    total_penalty = demand_penalty + emission_penalty

    return total_cost, total_penalty

# Setup DEAP toolbox.
toolbox = base.Toolbox()
toolbox.register("attr_int", random.randint, 0, 100)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, n=n_vars)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=100, indpb=0.1)
toolbox.register("select", tools.selNSGA2)

def main():
    random.seed(42)
    pop = toolbox.population(n=200)
    NGEN = 50
    MU = 200
    LAMBDA = 100
    CXPB = 0.7
    MUTPB = 0.2

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean, axis=0)
    stats.register("min", np.min, axis=0)
    
    hof = tools.ParetoFront()
    
    algorithms.eaMuPlusLambda(pop, toolbox, mu=MU, lambda_=LAMBDA,
                              cxpb=CXPB, mutpb=MUTPB, ngen=NGEN,
                              stats=stats, halloffame=hof, verbose=True)
    
    return pop, hof

# =============================================================================
# Run GA and Extract Results
# =============================================================================
if __name__ == "__main__":
    population, pareto_front = main()
    
    # For demonstration, select the first Pareto solution.
    best_solution = pareto_front[0]
    purchase_plan = { key: best_solution[i] for i, key in enumerate(solution_keys) }
    
    # Simulate fleet evolution (10-year lifetime)
    fleet = {yr: {} for yr in years}
    for yr in years:
        for vid in available_vehicle_ids:
            total = 0
            for y in years:
                if y <= yr and (y, vid) in purchase_plan and (yr - y) < 10:
                    total += purchase_plan[(y, vid)]
            fleet[yr][vid] = total

    # Unroll the plan into rows.
    rows = []
    # "Buy" rows: record when vehicles are purchased.
    for (yr, vid), num in purchase_plan.items():
        if num > 0:
            fuel = vehicle_props[vid].get("FuelType", "Unknown")
            rows.append({
                "Year": yr,
                "ID": vid,
                "Type": "Buy",
                "No_Vehicles": num,
                "Fuel": fuel
            })
    # "Use" rows: record operating fleet per year.
    for yr in years:
        for vid, count in fleet[yr].items():
            if count > 0:
                fuel = vehicle_props[vid].get("FuelType", "Unknown")
                rows.append({
                    "Year": yr,
                    "ID": vid,
                    "Type": "Use",
                    "No_Vehicles": count,
                    "Fuel": fuel
                })
    
    results_df = pd.DataFrame(rows)
    results_df.to_csv("fleet_operations.csv", index=False)
    
    # Compute final metrics.
    purchase_cost = sum(
        purchase_plan[(yr, vid)] * vehicle_props[vid]['Cost']
        for (yr, vid) in purchase_plan
    )
    fuel_cost = 0.0
    total_emissions = 0.0
    for yr in years:
        for vid, count in fleet[yr].items():
            fuel_cost += count * vehicle_props[vid]['Yearly_range'] * 0.5
            total_emissions += count * vehicle_props[vid]['Yearly_range'] * vehicle_props[vid]['Emission_rate']
    total_cost = purchase_cost + fuel_cost

    # Check demand satisfaction.
    demand_results = []
    bucket_order = {'D1': 1, 'D2': 2, 'D3': 3, 'D4': 4}
    for key, req in demand.items():
        d_year, d_size, d_distance = key
        provided = 0.0
        for vid, props in vehicle_props.items():
            if props['Size'] == d_size:
                if bucket_order[props['Distance']] >= bucket_order[d_distance]:
                    provided += fleet.get(d_year, {}).get(vid, 0) * props['Yearly_range']
        demand_results.append({
            "Year": d_year,
            "Size": d_size,
            "Distance": d_distance,
            "Demand (km)": req,
            "Provided (km)": provided,
            "Met": provided >= req
        })
    demand_df_out = pd.DataFrame(demand_results)
    demand_df_out.to_csv("demand_satisfaction.csv", index=False)

    # Print summary results.
    print("Final Total Cost:", total_cost)
    print("Total Carbon Emissions:", total_emissions)
    print("Demand Satisfaction Summary:")
    print(demand_df_out)


c:\Users\hjain\anaconda3\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMulti' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\hjain\anaconda3\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals	avg                            	min                            
0  	200   	[5.19549650e+09 2.36772125e+12]	[4.49131771e+09 1.87174123e+12]
1  	92    	[5.06064714e+09 2.28928017e+12]	[4.38150721e+09 1.87174123e+12]
2  	91    	[4.99271007e+09 2.23860186e+12]	[4.38150721e+09 1.87174123e+12]
3  	92    	[4.93259707e+09 2.19946433e+12]	[4.38150721e+09 1.87174123e+12]
4  	87    	[4.87377792e+09 2.16286620e+12]	[4.29110492e+09 1.87174123e+12]
5  	89    	[4.81665485e+09 2.12131623e+12]	[4.29110492e+09 1.87174123e+12]
6  	91    	[4.77342226e+09 2.08522781e+12]	[4.29110492e+09 1.87174123e+12]
7  	92    	[4.71013536e+09 2.05052578e+12]	[4.24862213e+09 1.87174123e+12]
8  	92    	[4.63110835e+09 2.01726313e+12]	[4.11838694e+09 1.85819344e+12]
9  	90    	[4.57292308e+09 1.98763628e+12]	[4.11838694e+09 1.82765122e+12]
10 	88    	[4.5314987e+09 1.9641018e+12]  	[4.11838694e+09 1.79220771e+12]
11 	89    	[4.49098716e+09 1.93916912e+12]	[4.11838694e+09 1.79220771e+12]
12 	87    	[4.44509310e+0